In [158]:
import librosa
import numpy as np
import torch
import pandas as pd

from src.constants import SAMPLE_RATE, WINDOW_LENGTH

HOP_LENGTH = 20
HOP_LENGTH = int(HOP_LENGTH / 1000 * SAMPLE_RATE)
SAMPLE_RATE = 16000

audio, _ = librosa.load("input/new_cut.wav", sr=SAMPLE_RATE)
audio = torch.from_numpy(audio).float()
audio_l = len(audio)
seq_len = 2.55
seq_len = int(seq_len * SAMPLE_RATE)

chunk_duration = 2.55
overlap = 1.5
chunk_samples = int(chunk_duration * SAMPLE_RATE)
chunk_overlap = int(chunk_duration * SAMPLE_RATE  // overlap)
divide_chunk = audio_l / chunk_samples
chunks = []


for i in range(0, audio_l - chunk_samples + 1, chunk_overlap):
    chunk = audio[i:i + chunk_samples]
    print('end =', i + chunk_samples)
    print('padleft = ', i)
    print('padright = ', len(audio)-i-chunk_samples)
    print('chunk shape = ', chunk.shape)
    print('-------')
    chunk = torch.nn.functional.pad(chunk, (i, audio_l-i-chunk_samples), 'constant', 0)
    chunk = np.pad(chunk, WINDOW_LENGTH // 2, mode="constant")        
    chunks.append(chunk)


datas = []

for chunk in chunks :
    data = []
    for i in range(len(chunk) // seq_len):
        begin_t = i * seq_len
        end_t = begin_t + seq_len + WINDOW_LENGTH
        data.append(
            chunk[begin_t:end_t],
        )
    chunk = chunk[end_t - WINDOW_LENGTH :]
    padding_size = seq_len + WINDOW_LENGTH - len(chunk)
    data.append(
        np.pad(chunk, (0, padding_size), mode='constant'),
    )
    datas.append(data)

datas = torch.from_numpy(np.array(datas)).float()
chunk_n = datas.shape[1]

end = 40800
padleft =  0
padright =  280736
chunk shape =  torch.Size([40800])
-------
end = 68000
padleft =  27200
padright =  253536
chunk shape =  torch.Size([40800])
-------
end = 95200
padleft =  54400
padright =  226336
chunk shape =  torch.Size([40800])
-------
end = 122400
padleft =  81600
padright =  199136
chunk shape =  torch.Size([40800])
-------
end = 149600
padleft =  108800
padright =  171936
chunk shape =  torch.Size([40800])
-------
end = 176800
padleft =  136000
padright =  144736
chunk shape =  torch.Size([40800])
-------
end = 204000
padleft =  163200
padright =  117536
chunk shape =  torch.Size([40800])
-------
end = 231200
padleft =  190400
padright =  90336
chunk shape =  torch.Size([40800])
-------
end = 258400
padleft =  217600
padright =  63136
chunk shape =  torch.Size([40800])
-------
end = 285600
padleft =  244800
padright =  35936
chunk shape =  torch.Size([40800])
-------
end = 312800
padleft =  272000
padright =  8736
chunk shape =  torch.Size([40800])
-

In [159]:
from src.model import E2E


model = torch.load('runs/Pitch_FL6_0/model.pt')
model.eval()
model = model.cuda()

In [160]:
results = []
overlap_results = []
for data in datas:
    data = data.cuda()
    with torch.no_grad():
        result = model(data)
        results.append(result[1].cpu().numpy())

results = torch.from_numpy(np.array(results)).float()

AssertionError: 

In [ ]:
from src.utils import to_local_average_cents

cents_pred_chunk = []

for result in results:
    print(result.shape)
    cents_pred = to_local_average_cents(result.view(-1,360).cpu().numpy(),None, 0 )
    cents_pred_chunk.append(cents_pred)

torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])
torch.Size([8, 128, 360])


In [ ]:
freqs = []
freq_l = cents_pred_chunk[0].shape[0]
freq_chunk_size = int(freq_l // chunk_n)
overlap_chunk =  int(freq_chunk_size // overlap)
for i, cents_pred in enumerate(cents_pred_chunk):
    freq = np.array(
        [10 * (2 ** (cent / 1200)) if cent else 0 for cent in cents_pred]
    )
    st = max(0,i*overlap_chunk)
    print(f'{st}: {st+freq_chunk_size}')
    freq_pad = np.pad(freq[st: st+freq_chunk_size], (st, freq_l - (st+freq_chunk_size)), mode="constant")
    freqs.append(freq_pad)

freqs = np.array(freqs)
freqs[freqs == 0] = np.nan
freq_result = np.nanmean(freqs, axis=0)

0: 128
85: 213
170: 298
255: 383
340: 468
425: 553
510: 638
595: 723
680: 808
765: 893


C:\Users\putte\AppData\Local\Temp\ipykernel_18772\2363046181.py:16: RuntimeWarning: Mean of empty slice
  freq_result = np.nanmean(freqs, axis=0)


In [ ]:
df = pd.DataFrame({"frequency": freq_result})

In [ ]:
df.to_csv('f0.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from scipy.io.wavfile import write

f0 = pd.read_csv("f0.csv")


def nco(fcw, sr):
    phase = 0
    phase_result = []
    for fcw_samp in fcw:
        ph_step = 2 * np.pi * fcw_samp * 1 / sr
        phase += ph_step
        phase_result.append(phase)
    return np.cos(phase_result)


step_size = 20
SAMPLE_RATE = 16000
quantum = (SAMPLE_RATE * step_size) // 1000
# sample_rate = quantum * (1000 // step_size)

fcw = np.zeros(quantum * f0.shape[0])

for i in range(f0.shape[0]):
    fcw[quantum * (i) : quantum * (i + 1)] = f0["frequency"][i]

output = nco(fcw, SAMPLE_RATE)



write("new_cut.wav", SAMPLE_RATE, output)